<a href="https://colab.research.google.com/github/SnehitReddy/AI_DCGAN_Art/blob/master/GANart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip uninstall torch torchvision -y
!pip install torchvision==0.1.6


Uninstalling torch-1.3.0+cu100:
  Successfully uninstalled torch-1.3.0+cu100
Uninstalling torchvision-0.4.1+cu100:
  Successfully uninstalled torchvision-0.4.1+cu100
ERROR: fastai 1.0.59 requires torch>=1.0.0, which is not installed.


In [0]:
%cd drive/My\ Drive/MLWork1

/content/drive/My Drive/MLWork1


In [0]:
!wget https://download.pytorch.org/whl/cu90/torch-0.3.1-cp36-cp36m-linux_x86_64.whl

--2019-11-06 03:24:43--  https://download.pytorch.org/whl/cu90/torch-0.3.1-cp36-cp36m-linux_x86_64.whl
Resolving download.pytorch.org (download.pytorch.org)... 13.224.2.116, 13.224.2.123, 13.224.2.121, ...
Connecting to download.pytorch.org (download.pytorch.org)|13.224.2.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 547781372 (522M) [binary/octet-stream]
Saving to: ‘torch-0.3.1-cp36-cp36m-linux_x86_64.whl’

torch-0.3.1-cp36-cp 100%[===================>] 522.40M  46.2MB/s    in 11s     

2019-11-06 03:24:55 (47.5 MB/s) - ‘torch-0.3.1-cp36-cp36m-linux_x86_64.whl’ saved [547781372/547781372]



In [0]:
!pip install ./torch-0.3.1-cp36-cp36m-linux_x86_64.whl

Processing ./torch-0.3.1-cp36-cp36m-linux_x86_64.whl
ERROR: fastai 1.0.59 has requirement torch>=1.0.0, but you'll have torch 0.3.1 which is incompatible.


In [0]:
!python ./DCGAN/make_gif.py

# **Readme**


---

This project attempts to train a Generative Adversarial Neural network to create images people would classify as artwork






In [0]:
with open('./movie.gif','rb') as f:
    display(Image(data=f.read(), format='png'))

In [0]:
with open('./art.gif','rb') as f:
    display(Image(data=f.read(), format='png'))

## How to run

---


This notebook has been setup to be compatible with the requirements of the network. If you wish to run this on your local machine, configure Pytorch to run with GPU



## Datasets

---


This network has been teseted on the Art Images (1) (https://www.kaggle.com/thedownhill/art-images-drawings-painting-sculpture-engraving), Danbooru 2018 (2) (https://www.gwern.net/Danbooru2018), and the Anime Faces (3) dataset at http://zhuanlan.zhihu.com/p/24767059 datasets



#Imports

In [0]:
import torch
from torch.autograd import Variable
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import imageio


#Constants

In [0]:
image_size = 64
G_input_dim = 100
G_output_dim = 3
D_input_dim = 3
D_output_dim = 1
num_filters = [1024, 512, 256, 128]

learning_rate = 0.0002
betas = (0.5, 0.999)
batch_size = 128
num_epochs = 100
data_dir = './DCGAN/processed_data/'
save_dir = './DCGAN/results/'

#Set Up DataStream

In [0]:
transform = transforms.Compose([transforms.Scale(image_size),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])

celebA_data = dsets.ImageFolder(data_dir, transform=transform)

data_loader = torch.utils.data.DataLoader(dataset=celebA_data,
                                          batch_size=batch_size,
                                          shuffle=True)


#Auxiliary Functions

###Denormalization

In [0]:


def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)



#Generator Model

In [0]:
class Generator(torch.nn.Module):
    def __init__(self, input_dim, num_filters, output_dim):
        super(Generator, self).__init__()

        # Hidden layers
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Convolutional Transpose layer
            if i == 0:
                convt = torch.nn.ConvTranspose2d(input_dim, num_filters[i], kernel_size=4, stride=1, padding=0)
            else:
                convt = torch.nn.ConvTranspose2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

            convt_name = 'convt' + str(i + 1)
            self.hidden_layer.add_module(convt_name, convt)

            # Initializer
            torch.nn.init.normal(convt.weight, mean=0.0, std=0.02)
            torch.nn.init.constant(convt.bias, 0.0)

            # Batch normalization
            bn_name = 'bn' + str(i + 1)
            self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

            # Activation
            act_name = 'act' + str(i + 1)
            self.hidden_layer.add_module(act_name, torch.nn.ReLU())

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Convolutional Transpose layer
        out = torch.nn.ConvTranspose2d(num_filters[i], output_dim, kernel_size=4, stride=2, padding=1)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Tanh())

    def forward(self, x):
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out


#Discriminator model

In [0]:

class Discriminator(torch.nn.Module):
    def __init__(self, input_dim, num_filters, output_dim):
        super(Discriminator, self).__init__()

        # Hidden layers
        self.hidden_layer = torch.nn.Sequential()
        for i in range(len(num_filters)):
            # Convolutional layer
            if i == 0:
                conv = torch.nn.Conv2d(input_dim, num_filters[i], kernel_size=4, stride=2, padding=1)
            else:
                conv = torch.nn.Conv2d(num_filters[i-1], num_filters[i], kernel_size=4, stride=2, padding=1)

            conv_name = 'conv' + str(i + 1)
            self.hidden_layer.add_module(conv_name, conv)

            # Initializer
            torch.nn.init.normal(conv.weight, mean=0.0, std=0.02)
            torch.nn.init.constant(conv.bias, 0.0)

            # Batch normalization
            if i != 0:
                bn_name = 'bn' + str(i + 1)
                self.hidden_layer.add_module(bn_name, torch.nn.BatchNorm2d(num_filters[i]))

            # Activation
            act_name = 'act' + str(i + 1)
            self.hidden_layer.add_module(act_name, torch.nn.LeakyReLU(0.2))

        # Output layer
        self.output_layer = torch.nn.Sequential()
        # Convolutional layer
        out = torch.nn.Conv2d(num_filters[i], output_dim, kernel_size=4, stride=1, padding=0)
        self.output_layer.add_module('out', out)
        # Initializer
        torch.nn.init.normal(out.weight, mean=0.0, std=0.02)
        torch.nn.init.constant(out.bias, 0.0)
        # Activation
        self.output_layer.add_module('act', torch.nn.Sigmoid())

    def forward(self, x):
        h = self.hidden_layer(x)
        out = self.output_layer(h)
        return out


#Visualizations

##Plot losses

In [0]:
def plot_loss(d_losses, g_losses, num_epoch, save=False, save_dir='./DCGAN/results/', show=False):
    fig, ax = plt.subplots()
    ax.set_xlim(0, num_epochs)
    ax.set_ylim(0, max(np.max(g_losses), np.max(d_losses))*1.1)
    plt.xlabel('Epoch {0}'.format(num_epoch + 1))
    plt.ylabel('Loss values')
    plt.plot(d_losses, label='Discriminator')
    plt.plot(g_losses, label='Generator')
    plt.legend()

    # save plot
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'losses_epoch_{:d}'.format(num_epoch + 1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()


##Plot generated image

In [0]:
def plot_result(generator, noise, num_epoch, save=False, save_dir='./DCGAN/results/', show=False, fig_size=(5, 5)):
    generator.eval()

    noise = Variable(noise.cuda(), volatile=True)
    gen_image = generator(noise)
    gen_image = denorm(gen_image)

    generator.train()

    n_rows = np.sqrt(noise.size()[0]).astype(np.int32)
    n_cols = np.sqrt(noise.size()[0]).astype(np.int32)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    for ax, img in zip(axes.flatten(), gen_image):
        ax.axis('off')
        ax.set_adjustable('box')
        img = (((img - img.min()) * 255) / (img.max() - img.min())).cpu().data.numpy().transpose(1, 2, 0).astype(np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch+1)
    fig.text(0.5, 0.04, title, ha='center')

    # save image
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'image_epoch_{:d}'.format(num_epoch+1) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()


#Driver Code

##Models

In [0]:

G = Generator(G_input_dim, num_filters, G_output_dim)
D = Discriminator(D_input_dim, num_filters[::-1], D_output_dim)
G.cuda()
D.cuda()


##Loss Funtion

In [0]:

criterion = torch.nn.BCELoss()

##Optimizers

In [0]:
G_optimizer = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
D_optimizer = torch.optim.Adam(D.parameters(), lr=learning_rate, betas=betas)


D_avg_losses = []
G_avg_losses = []


## Samples for GIF

In [0]:

num_test_samples = 5*5
fixed_noise = torch.randn(num_test_samples, G_input_dim).view(-1, G_input_dim, 1, 1)


#Training Loop

In [0]:
for epoch in range(num_epochs):
    D_losses = []
    G_losses = []

    # minibatch training
    for i, (images, _) in enumerate(data_loader):

        # image data
        mini_batch = images.size()[0]
        x_ = Variable(images.cuda())

        # labels
        y_real_ = Variable(torch.ones(mini_batch).cuda())
        y_fake_ = Variable(torch.zeros(mini_batch).cuda())

        # Train discriminator with real data
        D_real_decision = D(x_).squeeze()
        # print(D_real_decision, y_real_)
        D_real_loss = criterion(D_real_decision, y_real_)

        # Train discriminator with fake data
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())
        gen_image = G(z_)

        D_fake_decision = D(gen_image).squeeze()
        D_fake_loss = criterion(D_fake_decision, y_fake_)

        # Back propagation
        D_loss = D_real_loss + D_fake_loss
        D.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        # Train generator
        z_ = torch.randn(mini_batch, G_input_dim).view(-1, G_input_dim, 1, 1)
        z_ = Variable(z_.cuda())
        gen_image = G(z_)

        D_fake_decision = D(gen_image).squeeze()
        G_loss = criterion(D_fake_decision, y_real_)

        # Back propagation
        D.zero_grad()
        G.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        # loss values
        D_losses.append(D_loss.data[0])
        G_losses.append(G_loss.data[0])

        print('Epoch [%d/%d], Step [%d/%d], D_loss: %.4f, G_loss: %.4f'
              % (epoch+1, num_epochs, i+1, len(data_loader), D_loss.data[0], G_loss.data[0]))

    D_avg_loss = torch.mean(torch.FloatTensor(D_losses))
    G_avg_loss = torch.mean(torch.FloatTensor(G_losses))

    # avg loss values for plot
    D_avg_losses.append(D_avg_loss)
    G_avg_losses.append(G_avg_loss)

    plot_loss(D_avg_losses, G_avg_losses, epoch, save=True)

    # Show result for fixed noise
    plot_result(G, fixed_noise, epoch, save=True, fig_size=(5, 5))

    G_save_path = f'./DCGAN/models/gdcgan{epoch}.pth'
    D_save_path = f'./DCGAN/models/ddcgan{epoch}.pth'
    torch.save(G.state_dict(), G_save_path)
    torch.save(D.state_dict(), D_save_path)

####Run Trained Model

In [0]:
def plot_example(test_num, generator, noise, num_epoch, save=False, save_dir='./DCGAN/results/', show=False, fig_size=(5, 5)):
    generator.eval()

    noise = Variable(noise.cuda(), volatile=True)
    gen_image = generator(noise)
    gen_image = denorm(gen_image)

    n_rows = np.sqrt(noise.size()[0]).astype(np.int32)
    n_cols = np.sqrt(noise.size()[0]).astype(np.int32)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=fig_size)
    for ax, img in zip(axes.flatten(), gen_image):
        ax.axis('off')
        ax.set_adjustable('box')
        img = (((img - img.min()) * 255) / (img.max() - img.min())).cpu().data.numpy().transpose(1, 2, 0).astype(np.uint8)
        ax.imshow(img, cmap=None, aspect='equal')
    plt.subplots_adjust(wspace=0, hspace=0)
    title = 'Epoch {0}'.format(num_epoch+1)
    fig.text(0.5, 0.04, title, ha='center')

    # save image
    if save:
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)
        save_fn = save_dir + 'image_test_{:d}'.format(test_num) + '.png'
        plt.savefig(save_fn)

    if show:
        plt.show()
    else:
        plt.close()


In [0]:
PATH = ''
G.load_state_dict(torch.load(PATH))

xzy_noise= torch.randn(1, G_input_dim).view(-1, G_input_dim, 1, 1)
xzy_noise = Variable(xzy_noise.cuda())
gen_image = G(xzy_noise)
plot_example(69, G, fixed_noise, epoch, save=True, fig_size=(5, 5))









